In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os
import numpy as np
import pandas as pd
import anndata as ad 
import squidpy as sq
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sb
import scvi 
import torch #pytorch
import runpy
from datetime import datetime
import celltypist
from celltypist import models

from os import sys
sys.path.append('/tmp/work/Visium/sean/brca_scripts')
from visiumPipeline import processVisium
from applySignatures import cosineSignatures


sc.set_figure_params(figsize=(4,4))
sc.set_figure_params(facecolor="white", figsize=(8, 8))
sc.settings.verbosity = 3

#gettting model for celltypist
models.download_models(force_update = True)
model = models.Model.load(model = 'Immune_All_High.pkl')

#define sample poths, sample names and emptry dictionary

samples = ['2018838/PRE-01','2018838/PRE-02','2018838/POST-03','2018838/POST-05',
           '2018839/PRE-04','2018839/PRE-05','2018839/POST-06','2018839/POST-11']

sample_names = [os.path.basename(sample) for sample in samples]
visium_samples = variables = {name: None for name in sample_names} #variable names associated with emptry values in dictionary for storing data later

/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/scvi/__init__.py:31: DeprecationWarning: scvi is deprecated, please uninstall scvi via `pip uninstall scvi` and install the new scvi-tools package at github.com/YosefLab/scvi-tools
  warnings.warn(deprecation_msg, DeprecationWarning)
/opt/conda/lib/python3.10/site-packages/scanpy/_settings.py:450: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_forma

In [2]:
# construct the pseduobulk reference
start_time = datetime.now()
result = runpy.run_path('/tmp/work/Visium/sean/brca_scripts/acquirePseudoBulkRef.py')
end_time = datetime.now()
execution_time = end_time - start_time
print('\n')
print(f"Execution time: {execution_time}")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


normalizing counts per cell
    finished (0:00:01)
If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
    finished (0:00:02)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:01)
computing neighbors
    using 'X_pca' with n_pcs = 50


2024-08-13 17:18:14.860679: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:24)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:01:03)
normalizing counts per cell
    finished (0:00:00)


Execution time: 0:04:23.819774


In [3]:
#process each of the samples

start_time = datetime.now()

for i in range(len(samples)):
    print('\n')
    print('Beginng to process '+sample_names[i])
    print('\n')
    visium_samples[sample_names[i]] = processVisium(samples[i],model)    
    
end_time = datetime.now()
execution_time = end_time - start_time
print('\n')
print(f"Execution time: {execution_time}")



Beginng to process PRE-01


reading /tmp/work/Visium/2018838/PRE-01/filtered_feature_bc_matrix.h5
 (0:00:01)
filtered out 60 cells that have less than 1000 counts
filtered out 45 genes that are detected in less than 5 cells
normalizing counts per cell
    finished (0:00:00)
If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:00)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:01)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:04)
running Leiden cluster

🔬 Input data has 4668 cells and 18040 genes
🔗 Matching reference genes in the model
🧬 5047 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!
👀 Detected a neighborhood graph in the input object, will run over-clustering on the basis of it
⛓️ Over-clustering input data with resolution set to 5


running Leiden clustering
    finished: found 53 clusters and added
    'over_clustering', the cluster labels (adata.obs, categorical) (0:00:00)


🗳️ Majority voting the predictions
✅ Majority voting done!




Beginng to process PRE-02


reading /tmp/work/Visium/2018838/PRE-02/filtered_feature_bc_matrix.h5
 (0:00:01)
filtered out 98 cells that have less than 1000 counts
filtered out 58 genes that are detected in less than 5 cells
normalizing counts per cell
    finished (0:00:00)
If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:00)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:00)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:04)
running Leiden cluster

...



Beginng to process POST-11


reading /tmp/work/Visium/2018839/POST-11/filtered_feature_bc_matrix.h5
 (0:00:00)
filtered out 1500 cells that have less than 1000 counts
filtered out 72 genes that are detected in less than 5 cells
normalizing counts per cell
    finished (0:00:00)
If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:00)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:00)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:03)
running Leiden clu

🔬 Input data has 3137 cells and 18013 genes
🔗 Matching reference genes in the model
🧬 5045 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!
👀 Detected a neighborhood graph in the input object, will run over-clustering on the basis of it
⛓️ Over-clustering input data with resolution set to 5


running Leiden clustering
    finished: found 62 clusters and added
    'over_clustering', the cluster labels (adata.obs, categorical) (0:00:00)


🗳️ Majority voting the predictions
✅ Majority voting done!




Execution time: 0:01:18.879965


In [4]:
# apply signature values to each of the samples

start_time = datetime.now()

pb_df = pd.read_pickle('wu_2021_pseudobulk.pkl')

for i in range(len(sample_names)):
    print('\n')
    print('Now applying cosine signature to '+sample_names[i])
    print('\n')
    visium_samples[sample_names[i]] = cosineSignatures(visium_samples[sample_names[i]],pb_df)    
    
end_time = datetime.now()
execution_time = end_time - start_time
print('\n')
print(f"Execution time: {execution_time}")



Now applying cosine signature to PRE-01


Valid columns: 905
Missing columns: 95


Now applying cosine signature to PRE-02


Valid columns: 935
Missing columns: 65


Now applying cosine signature to POST-03


Valid columns: 904
Missing columns: 96


Now applying cosine signature to POST-05


Valid columns: 898
Missing columns: 102


Now applying cosine signature to PRE-04


Valid columns: 902
Missing columns: 98


Now applying cosine signature to PRE-05


Valid columns: 926
Missing columns: 74


Now applying cosine signature to POST-06


Valid columns: 895
Missing columns: 105


Now applying cosine signature to POST-11


Valid columns: 902
Missing columns: 98


Execution time: 0:00:22.599274


In [26]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import scanpy as sc
import squidpy as sq
import warnings

cos_metrics =  ['B-cells_cos_sim', 'CAFs_cos_sim', 'Cancer Epithelial_cos_sim', 'Endothelial_cos_sim', 'Myeloid_cos_sim', 'Normal Epithelial_cos_sim', 'PVL_cos_sim', 'Plasmablasts_cos_sim', 'T-cells_cos_sim']

# Define sample names and split into pre and post
sample_names = ['PRE-01', 'PRE-02', 'POST-03', 'POST-05', 'PRE-04', 'PRE-05', 'POST-06', 'POST-11']
pre_samples = [s for s in sample_names if s.startswith('PRE-')]
post_samples = [s for s in sample_names if s.startswith('POST-')]

warnings.filterwarnings("ignore", category=UserWarning, message=".*MatplotlibDeprecationWarning.*")
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Define figure directory and ensure it exists
figure_dir = os.path.join(os.getcwd(), 'figures')
os.makedirs(figure_dir, exist_ok=True)  # Create directory if it doesn't exist

for metric in cos_metrics:
    pdf_path = os.path.join(figure_dir, f'{metric}.pdf')
    pdf_pages = PdfPages(pdf_path)
    
    vmin = 0.2
    vmax = 0.6
    current_cmap = 'viridis'
    
    column_labels = ['Pre-Menopausal', 'Post-Menopausal']
    
    for i in range(4):
        # Ensure all categories are present on axes for celltypist majority voting
        full_categories = ['B cells', 'Endothelial cells', 'Epithelial cells', 'Fibroblasts', 'Macrophages', 'Plasma cells']
        visium_samples[pre_samples[i]].uns.pop('majority_voting_colors', None)
        visium_samples[post_samples[i]].uns.pop('majority_voting_colors', None)
        visium_samples[pre_samples[i]].obs['majority_voting'] = visium_samples[pre_samples[i]].obs['majority_voting'].cat.set_categories(full_categories)
        visium_samples[post_samples[i]].obs['majority_voting'] = visium_samples[post_samples[i]].obs['majority_voting'].cat.set_categories(full_categories)
    
        fig, axes = plt.subplots(3, 2, figsize=(12, 12))  # Create a grid of subplots

        #Plot the images on their own

        sq.pl.spatial_scatter(visium_samples[pre_samples[i]],ax=axes[0, 0])
        axes[0, 0].set_xlabel('')
        axes[0, 0].set_xlabel('')
        sq.pl.spatial_scatter(visium_samples[post_samples[i]],ax=axes[0, 1])
        axes[0, 1].set_xlabel('')
        axes[0, 1].set_xlabel('')

    
        # Plot each spatial scatter
        sq.pl.spatial_scatter(visium_samples[pre_samples[i]], color="majority_voting", alpha=1.0, ax=axes[1, 0],legend_loc='none')
        sq.pl.spatial_scatter(visium_samples[post_samples[i]], color="majority_voting", alpha=1.0, ax=axes[1, 1])
        axes[1, 1].set_ylabel('')
        
        # sq.pl.spatial_scatter(visium_samples[pre_samples[i]], color=metric, alpha=1.0, ax=axes[1, 0], cmap=current_cmap, colorbar = False)
        # sq.pl.spatial_scatter(visium_samples[post_samples[i]], color=metric, alpha=1.0, ax=axes[1, 1], cmap=current_cmap, colorbar = False)

        sq.pl.spatial_scatter(visium_samples[pre_samples[i]], color=metric, alpha=1.0, ax=axes[2, 0], vmin=vmin, vmax=vmax, cmap=current_cmap, colorbar = False)
        sq.pl.spatial_scatter(visium_samples[post_samples[i]], color=metric, alpha=1.0, ax=axes[2, 1], vmin=vmin, vmax=vmax, cmap=current_cmap)
        axes[2, 1].set_ylabel('')
    
        # Set column labels using fig.text
        fig.text(0.27, 0.94, column_labels[0], ha='center', fontsize=16, weight='bold')
        fig.text(0.66, 0.94, column_labels[1], ha='center', fontsize=16, weight='bold')
    
        # Adjust layout to fit labels and minimize space
        plt.subplots_adjust(top=0.6)  # Adjust top margin to fit column labels
        plt.tight_layout(rect=[0, 0, 1, 0.94])  # Adjust layout to fit content within the figure
    
        pdf_pages.savefig(fig)  # Save the figure to the PDF
        #plt.show()  # Display the figure in the notebook
        plt.close(fig)  # Close the figure to free memory
    
    pdf_pages.close()